In [1]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
%%capture
!pip install open_clip_torch

In [4]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!git checkout -q 57535c5a79791cb76e36fdb64975271354f10251
!pip install -q -e .

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 401 (delta 128), reused 120 (delta 120), pack-reused 237
Receiving objects: 100% (401/401), 12.84 MiB | 17.94 MiB/s, done.
Resolving deltas: 100% (205/205), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.3 MB/s eta 0:00:00


In [5]:
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

/content
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-tfws8z5q
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-tfws8z5q
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36586 sha256=81f851d7be518d678a7a6808eb7691c81c1f82a58defe5646ebf0f40e1d3ab28
  Stored in directory: /tmp/pip-ephem-wheel-cache-bz6vi9fa/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [6]:
!pip uninstall -y supervision
!pip install -q supervision==0.6.0

import supervision as sv
print(sv.__version__)

Found existing installation: supervision 0.4.0
Uninstalling supervision-0.4.0:
  Successfully uninstalled supervision-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
groundingdino 0.1.0 requires supervision==0.4.0, but you have supervision 0.6.0 which is incompatible.
0.6.0


In [7]:
import os

GROUNDING_DINO_CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [8]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [9]:
GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [10]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

/content
/content/weights


In [11]:
SAM_CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(SAM_CHECKPOINT_PATH, "; exist:", os.path.isfile(SAM_CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [12]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import Model

grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

/content/GroundingDINO


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


In [13]:
from segment_anything import sam_model_registry, SamPredictor

SAM_ENCODER_VERSION = "vit_h"

sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_CHECKPOINT_PATH).to('cuda')
sam_predictor = SamPredictor(sam)

In [14]:
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision.io import read_image
from torch.utils.data import Dataset
import pandas as pd
import torchvision.transforms as T
import torchvision.transforms.functional as F
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import open_clip
from typing import List
import math
import cv2
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
%%capture
!unzip /content/gdrive/MyDrive/test-archive.zip -d /content/testing-dataset

In [17]:
!mv "/content/testing-dataset/development_test_data/gallery" "/content/"
!mv "/content/testing-dataset/development_test_data/queries" "/content/"

In [18]:
!mkdir "/content/original_datasets"
!mv "/content/testing-dataset/development_test_data/gallery.csv" "/content/original_datasets/"
!mv "/content/testing-dataset/development_test_data/queries.csv" "/content/original_datasets/"

In [19]:
!rm -rf "/content/testing-dataset"

In [20]:
!mkdir '/content/image_to_text_results/'
!cp '/content/gdrive/MyDrive/prompt_What-is-the-product-category_all_version.csv' '/content/image_to_text_results/'
!cp '/content/gdrive/MyDrive/prompt_What-is-the-product-category_gallery_version.csv' '/content/image_to_text_results/'
!cp '/content/gdrive/MyDrive/prompt_What-is-the-product-category.csv' '/content/image_to_text_results/'

In [21]:
%%capture
!unzip '/content/gdrive/MyDrive/gallery_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25.zip' -d '/'
!unzip '/content/gdrive/MyDrive/query_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25.zip' -d '/'

In [22]:
!mkdir '/content/pretrained'
!cp '/content/gdrive/MyDrive/visual-product-recognition-pre-trained/model1.pt' '/content/pretrained/'
!cp '/content/gdrive/MyDrive/visual-product-recognition-pre-trained/model2.pt' '/content/pretrained/'
!cp '/content/gdrive/MyDrive/visual-product-recognition-pre-trained/model3.pt' '/content/pretrained/'

In [23]:
masked_similarity = pd.read_csv('/content/gallery_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25/gallery_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25.csv', low_memory=False)
masked_similarity.head()

,index,image_tensor_path,masked_image_tensor_path,inverse_masked_image_tensor_path,label,image_path,masked_similarity,inverse_masked_similarity
0,0,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,5019,gallery/ambitious-tough-teal-from-asgard.jpg,12.038039,18.042643
1,1,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,5008,gallery/fine-shrewd-oarfish-of-genius.jpg,5.544218,18.558239
2,2,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,9,gallery/bold-nickel-gecko-of-reputation.jpg,1.164761,14.268379
3,3,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,3589,gallery/marvellous-uber-boobook-of-lightning.jpg,4.094361,12.754694
4,4,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,3656,gallery/bouncy-economic-agama-of-honeydew.jpg,2.484441,15.336771


In [24]:
detection_category = pd.read_csv('/content/image_to_text_results/prompt_What-is-the-product-category_all_version.csv', low_memory=False)
detection_category.head()

,index,label,category,count
0,0,5019,shoes,7
1,1,5008,shoes,8
2,2,9,basketball,5
3,3,3589,sunglasses,3
4,4,3656,sunglasses,4


In [25]:
gallery = pd.read_csv("/content/original_datasets/gallery.csv", low_memory=False)
gallery.head()

,seller_img_id,img_path,product_id
0,0,gallery/ambitious-tough-teal-from-asgard.jpg,5019
1,1,gallery/fine-shrewd-oarfish-of-genius.jpg,5008
2,2,gallery/bold-nickel-gecko-of-reputation.jpg,9
3,3,gallery/marvellous-uber-boobook-of-lightning.jpg,3589
4,4,gallery/bouncy-economic-agama-of-honeydew.jpg,3656


In [26]:
def get_mask_or_inverse_mask_to_use(img_path):
  ms = (masked_similarity.loc[masked_similarity['image_path'] == img_path, 'masked_similarity'].values)[0]
  ims = (masked_similarity.loc[masked_similarity['image_path'] == img_path, 'inverse_masked_similarity'].values)[0]
  if ms <= ims:
    return 1
  else:
    return 0

img_df = gallery
df = pd.DataFrame(columns=['img1', 'img2', 'label', 'img1_masked', 'img2_masked', 'img1_category', 'img2_category'])
product_ids = gallery['product_id'].unique()
row_count = 0
id_count = {1:0, 2:0, 3:0, 4:0}
for product_id in product_ids:
  df1 = img_df.loc[img_df['product_id'] == product_id]
  img_category = (detection_category.loc[detection_category['label'] == product_id, 'category'].values)[0]
  img_path_index = df1.columns.get_loc('img_path')
  if len(df1) == 2:
    row_count += 1
    id_count[2] += 1
    img1 = df1.iloc[0, img_path_index]
    img2 = df1.iloc[1, img_path_index]
    df.loc[len(df)] = [img1, img2, 1, get_mask_or_inverse_mask_to_use(img1), get_mask_or_inverse_mask_to_use(img2), img_category, img_category]
  elif len(df1) == 3:
    img1 = df1.iloc[0, img_path_index]
    img2 = df1.iloc[1, img_path_index]
    img3 = df1.iloc[2, img_path_index]
    row_count += 2
    id_count[3] += 1
    df.loc[len(df)] = [img1, img2, 1, get_mask_or_inverse_mask_to_use(img1), get_mask_or_inverse_mask_to_use(img2), img_category, img_category]
    df.loc[len(df)] = [img2, img3, 1, get_mask_or_inverse_mask_to_use(img2), get_mask_or_inverse_mask_to_use(img3), img_category, img_category]
  elif len(df1) > 3:
    id_count[4] += 1
    img1 = df1.iloc[0, img_path_index]
    img2 = df1.iloc[1, img_path_index]
    img3 = df1.iloc[2, img_path_index]
    img4 = df1.iloc[3, img_path_index]
    row_count += 2
    df.loc[len(df)] = [img1, img2, 1, get_mask_or_inverse_mask_to_use(img1), get_mask_or_inverse_mask_to_use(img2), img_category, img_category]
    df.loc[len(df)] = [img3, img4, 1, get_mask_or_inverse_mask_to_use(img3), get_mask_or_inverse_mask_to_use(img4), img_category, img_category]
  else:
    id_count[1] += 1
  df2 = img_df.loc[img_df['product_id'] != product_id]
  if len(df1) < 4:
    img1 = df1.iloc[0, img_path_index]
    count = 0
    for index, row in df2.iterrows():
      if count >= 4:
        break
      if ((df['img1'] == img1) & (df['img2'] == row['img_path'])).any() or ((df['img1'] == row['img_path']) & (df['img2'] == img1)).any():
        continue
      else:
        img2 = row['img_path']
        row_count += 1
        img2_category = (detection_category.loc[detection_category['label'] == row['product_id'], 'category'].values)[0]
        df.loc[len(df)] = [img1, img2, 0, get_mask_or_inverse_mask_to_use(img1), get_mask_or_inverse_mask_to_use(img2), img_category, img2_category]
        count += 1
  else:
    img1 = df1.iloc[0, img_path_index]
    img2 = df1.iloc[1, img_path_index]
    count = 0
    for index, row in df2.iterrows():
      if count >= 4:
        break
      if count >= 2:
        img1 = img2
      if ((df['img1'] == img1) & (df['img2'] == row['img_path'])).any() or ((df['img1'] == row['img_path']) & (df['img2'] == img1)).any():
        continue
      else:
        img3 = row['img_path']
        row_count += 1
        img2_category = (detection_category.loc[detection_category['label'] == row['product_id'], 'category'].values)[0]
        df.loc[len(df)] = [img1, img3, 0, get_mask_or_inverse_mask_to_use(img1), get_mask_or_inverse_mask_to_use(img3), img_category, img2_category]
        count += 1

df.to_csv('/content/train_validation_dataset.csv')
print('row_count: ', row_count)
print(len(df.loc[df['label'] == 1]))
print(id_count)

row_count:  2871
435
{1: 284, 2: 215, 3: 90, 4: 20}


In [27]:
print('shape: ', df.shape)
display(df.head())

shape:  (2871, 7)


,img1,img2,label,img1_masked,img2_masked,img1_category,img2_category
0,gallery/ambitious-tough-teal-from-asgard.jpg,gallery/busy-aspiring-oryx-of-peace.jpg,1,1,1,shoes,shoes
1,gallery/busy-aspiring-oryx-of-peace.jpg,gallery/vigorous-upbeat-barracuda-of-strength.jpg,1,1,1,shoes,shoes
2,gallery/ambitious-tough-teal-from-asgard.jpg,gallery/fine-shrewd-oarfish-of-genius.jpg,0,1,1,shoes,shoes
3,gallery/ambitious-tough-teal-from-asgard.jpg,gallery/bold-nickel-gecko-of-reputation.jpg,0,1,1,shoes,basketball
4,gallery/ambitious-tough-teal-from-asgard.jpg,gallery/marvellous-uber-boobook-of-lightning.jpg,0,1,1,shoes,sunglasses


In [28]:
df1 = df.loc[df['label'] == 1]
df2 = df.loc[df['label'] == 0]

msk1 = np.random.rand(len(df1)) < 0.8
train1 = df1[msk1]
validation1 = df1[~msk1]

msk2 = np.random.rand(len(df2)) < 0.8
train2 = df2[msk2]
validation2 = df2[~msk2]

train = pd.concat([train1, train2], axis=0)
validation = pd.concat([validation1, validation2], axis=0)

train.reset_index(inplace=True)
train.drop(['index'], inplace=True, axis=1)
train.to_csv('/content/train_dataset_for_segmentation_model.csv')

validation.reset_index(inplace=True)
validation.drop(['index'], inplace=True, axis=1)
validation.to_csv('/content/validation_dataset_for_segmentation_model.csv')

print('Train\n====================================')
print('shape: ', train.shape)
display(train.head())

print('Validation\n====================================')
print('shape: ', validation.shape)
display(validation.head())

Train
shape:  (2302, 7)


,img1,img2,label,img1_masked,img2_masked,img1_category,img2_category
0,gallery/ambitious-tough-teal-from-asgard.jpg,gallery/busy-aspiring-oryx-of-peace.jpg,1,1,1,shoes,shoes
1,gallery/busy-aspiring-oryx-of-peace.jpg,gallery/vigorous-upbeat-barracuda-of-strength.jpg,1,1,1,shoes,shoes
2,gallery/fine-shrewd-oarfish-of-genius.jpg,gallery/alluring-quiet-peacock-of-renovation.jpg,1,1,1,shoes,shoes
3,gallery/dainty-stork-of-hypothetical-health.jpg,gallery/kind-opalescent-lion-of-adventure.jpg,1,1,1,shoes,shoes
4,gallery/bold-nickel-gecko-of-reputation.jpg,gallery/colorful-mutant-boobook-of-popularity.jpg,1,1,1,basketball,basketball


Validation
shape:  (569, 7)


,img1,img2,label,img1_masked,img2_masked,img1_category,img2_category
0,gallery/enigmatic-masked-dragonfly-of-might.jpg,gallery/natural-smiling-mackerel-of-storm.jpg,1,1,1,shoes,shoes
1,gallery/natural-smiling-mackerel-of-storm.jpg,gallery/sceptical-attractive-deer-of-control.jpg,1,1,0,shoes,shoes
2,gallery/dramatic-olivine-silkworm-of-radiance.jpg,gallery/elated-ingenious-sturgeon-of-blizzard.jpg,1,1,1,sunglasses,sunglasses
3,gallery/meteoric-bright-cassowary-of-bliss.jpg,gallery/satisfied-married-ara-of-revolution.jpg,1,1,1,sunglasses,sunglasses
4,gallery/burrowing-carmine-shrew-of-growth.jpg,gallery/tacky-berserk-earwig-of-luxury.jpg,1,1,1,shoes,shoes


In [29]:
def enhance_class_name(class_names: List[str]) -> List[str]:
    return [
        f"all {class_name}s"
        for class_name
        in class_names
    ]

def segment(sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(
            box=box,
            multimask_output=True
        )
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)

def detect_masks(img_path, category):
    if category is not None:
      CLASSES = [category]
    else:
      CLASSES = ['products']
    BOX_TRESHOLD = 0.4
    TEXT_TRESHOLD = 0.25
    image = cv2.imread(img_path)
    detections = grounding_dino_model.predict_with_classes(
        image=image,
        classes=enhance_class_name(class_names=CLASSES),
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )
    detections = detections[detections.class_id != None]
    detections.mask = segment(
        sam_predictor=sam_predictor,
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
        xyxy=detections.xyxy
    )
    return detections.mask, detections.xyxy

def get_mask_and_inverse_mask(img_path, category):
  masks, bboxes = detect_masks(img_path, category)
  image = read_image(img_path)

  if len(masks) >= 1:
    masks_np_arr = {}

    for i in range(len(masks)):
      mask = masks[i:i+1, :]
      mask = mask.astype(int)
      mask = np.repeat(mask, 3, axis = 0)
      # mask = np.where(mask==0, 0, 1)
      # mask = mask.astype('uint8')
      masks_np_arr[i] = mask

    final_mask = None
    if len(masks_np_arr) > 1:
      for key, mask in masks_np_arr.items():
        if final_mask is None:
          final_mask = mask
        else:
          final_mask = np.add(final_mask, mask)
      final_mask = np.where(final_mask > 0, 1, 0)
      final_inverse_mask = np.where(final_mask==0, 1, 0)
      # final_mask = final_mask.astype('uint8')
      # final_inverse_mask = final_inverse_mask.astype('uint8')
    else:
      final_mask = masks_np_arr[0]
      final_inverse_mask = np.where(final_mask==0, 1, 0)
      # final_inverse_mask = final_inverse_mask.astype('uint8')

    image_ = image.numpy()

    # final_mask = final_mask.astype('int64')
    final_mask = np.where(final_mask == 0, -1, 1)
    masked = np.multiply(image_, final_mask)
    masked = np.where(masked < 0, 240, masked)
    masked = masked.astype('uint8')

    # final_inverse_mask = final_inverse_mask.astype('int64')
    final_inverse_mask = np.where(final_inverse_mask == 0, -1, 1)
    inverse_masked = np.multiply(image_, final_inverse_mask)
    inverse_masked = np.where(inverse_masked < 0, 240, inverse_masked)
    inverse_masked = inverse_masked.astype('uint8')

    masked = torch.tensor(masked)
    inverse_masked = torch.tensor(inverse_masked)

    # calculate bounding box parameters for cropping
    x = []
    y = []
    for bbox in bboxes:
      x.append(bbox[0])
      x.append(bbox[2])
      y.append(bbox[1])
      y.append(bbox[3])
    min_x = math.floor(min(x))
    min_y = math.floor(min(y))
    max_x = math.ceil(max(x))
    max_y = math.ceil(max(y))

    return image, masked, inverse_masked, (min_x, min_y, max_x - min_x, max_y - min_y)
  else:
    return image, None, None, None

In [30]:
def get_img_color_convert(img_path):
  """
  This function returns an image in the format RGB when the path
  to the image is given

  Parameters
  ----------
  img_path: string
      The path to the image

  Returns
  -------
  RGB image of the image in the specified path
  """
  return cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

def get_img(img_path):
  """
  This function returns an image in the format RGB when the path
  to the image is given

  Parameters
  ----------
  img_path: string
      The path to the image

  Returns
  -------
  RGB image of the image in the specified path
  """
  return cv2.imread(img_path)

def draw_bounding_box_and_save(img_path, x, y, w, h):
  """
  This function draw a bounding box specified by x, y, w, h parameters
  and save the image in the '/content/temp-imgs/' folder

  Parameters
  ----------
  img_path: string
      The path to original image which the bounding box should be drawn
  x: integer
      x-coordinate of top-left corner of bounding box
  y: integer
      y-coordinate of top-left corner of bounding box
  w: integer
      width of bounding box in pixels
  h: integer
      height of bounding box in pixels
  """
  image = get_img(img_path)
  image = image[y:y+h, x:x+w]
  # cv2.rectangle(image, (x, y), (x + w, y + h), (0,255,255), 4)

  Path("/content/temp-imgs").mkdir(parents=True, exist_ok=True)

  filepath = '/content/temp-imgs/' + img_path.split('/')[-1]
  cv2.imwrite(filepath, image)
  return filepath

In [31]:
img_transform = T.Compose([T.ToPILImage(),
                           T.Resize(size=(224, 224),
                                    interpolation=T.InterpolationMode.BICUBIC,
                                    antialias=True),
                           T.ToTensor(),
                           T.Normalize(mean=(0.48145466, 0.4578275, 0.40821073),
                                       std=(0.26862954, 0.26130258, 0.27577711))])

In [32]:
class TrainValidationDataset(Dataset):
    def __init__(self, annotations_file, img_dir, indices,
                 transform=None,
                 crop_transform=None,
                 target_transform=None,
                 crop_to_bbox=False,
                 crop_to_bbox_detections=False):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.indices = indices
        self.transform = transform
        self.target_transform = target_transform
        self.crop_to_bbox = crop_to_bbox
        self.crop_to_bbox_detections = crop_to_bbox_detections
        self.crop_transform = crop_transform

    def __len__(self):
        return len(self.img_labels)
        # return 2

    def __getitem__(self, idx):
        img1_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, self.indices[0]])
        img2_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, self.indices[1]])
        label = self.img_labels.iloc[idx, self.indices[2]]
        img1_mask = self.img_labels.iloc[idx, self.indices[3]]
        img2_mask = self.img_labels.iloc[idx, self.indices[4]]
        img1_cat = self.img_labels.iloc[idx, self.indices[5]]
        img2_cat = self.img_labels.iloc[idx, self.indices[6]]

        if pd.isna(img1_cat):
          img1_cat = None
        if pd.isna(img2_cat):
          img2_cat = None

        image1, masked1, inverse_masked1, bbox1 = get_mask_and_inverse_mask('/content/' + img1_path, img1_cat)
        image2, masked2, inverse_masked2, bbox2 = get_mask_and_inverse_mask('/content/' + img2_path, img2_cat)

        if img1_mask == 0:
          masked1 = inverse_masked1

        if img2_mask == 0:
          masked2 = inverse_masked2

        if self.transform and not self.crop_to_bbox_detections:
            image1 = self.transform(image1)
            image2 = self.transform(image2)
            if masked1 is not None:
              masked1 = self.transform(masked1)
            if masked2 is not None:
              masked2 = self.transform(masked2)
        elif self.crop_transform and self.crop_to_bbox_detections:
            if masked1 is not None and bbox1 is not None:
              x = bbox1[0]
              y = bbox1[1]
              w = bbox1[2]
              h = bbox1[3]
              image1 = self.crop_transform(image1, x, y, w, h)
              masked1 = self.crop_transform(masked1, x, y, w, h)
            else:
              if self.transform:
                image1 = self.transform(image1)

            if masked2 is not None and bbox2 is not None:
              x = bbox2[0]
              y = bbox2[1]
              w = bbox2[2]
              h = bbox2[3]
              image2 = self.crop_transform(image2, x, y, w, h)
              masked2 = self.crop_transform(masked2, x, y, w, h)
            else:
              if self.transform:
                image2 = self.transform(image2)

        if self.target_transform:
            label = self.target_transform(label)

        if masked1 is None:
          masked1 = image1

        if masked2 is None:
          masked2 = image2

        return masked1, masked2, label

def image_transformation(image):
    images = img_transform(image)
    #images = img_transform(F.to_pil_image(image))
    return images

def image_transform_with_bbox_crop(image, x, y, w, h):
    images = F.crop(image, y, x, h, w)
    images = img_transform(images)
    return images

In [33]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

In [34]:
vit_model = open_clip.create_model_and_transforms('ViT-H-14', None)[0].visual
vit_model.load_state_dict(torch.load('/content/pretrained/model3.pt'))
vit_model

VisionTransformer(
  (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
)

In [35]:
for name, para in vit_model.named_parameters():
  print("-"*20)
  print(f"name: {name}")

--------------------
name: class_embedding
--------------------
name: positional_embedding
--------------------
name: proj
--------------------
name: conv1.weight
--------------------
name: ln_pre.weight
--------------------
name: ln_pre.bias
--------------------
name: transformer.resblocks.0.ln_1.weight
--------------------
name: transformer.resblocks.0.ln_1.bias
--------------------
name: transformer.resblocks.0.attn.in_proj_weight
--------------------
name: transformer.resblocks.0.attn.in_proj_bias
--------------------
name: transformer.resblocks.0.attn.out_proj.weight
--------------------
name: transformer.resblocks.0.attn.out_proj.bias
--------------------
name: transformer.resblocks.0.ln_2.weight
--------------------
name: transformer.resblocks.0.ln_2.bias
--------------------
name: transformer.resblocks.0.mlp.c_fc.weight
--------------------
name: transformer.resblocks.0.mlp.c_fc.bias
--------------------
name: transformer.resblocks.0.mlp.c_proj.weight
--------------------
name:

In [36]:
# Fully connected neural network with one hidden layer
class SegmentationModel(nn.Module):
    def __init__(self):
        super(SegmentationModel, self).__init__()

        vit_model = open_clip.create_model_and_transforms('ViT-H-14', None)[0].visual
        vit_model.load_state_dict(torch.load('/content/pretrained/model3.pt'))
        self.vit_model = vit_model

    def forward(self, x1, x2):
      emb1 = self.vit_model(x1)
      emb2 = self.vit_model(x2)
      return emb1, emb2

In [37]:
def criterion(x1, x2, label, margin: float = 40.0):
    """
    Computes Contrastive Loss
    """

    dist = torch.nn.functional.pairwise_distance(x1, x2, p=2)

    loss = (label) * torch.pow(dist, 2) + (1 - label) * torch.pow(torch.clamp(margin - dist, min=0.0), 2)
    loss = torch.mean(loss)

    return loss

In [38]:
# Make sure reproducability
np.random.seed(0)
torch.manual_seed(0)

# hyperparameters
num_epochs = 10
learning_rate = 0.001

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SegmentationModel().to(device)
for name, para in vit_model.named_parameters():
  if 'mlp.c_proj' in name:
    para.requires_grad = True
  else:
    para.requires_grad = False

# Loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Early stopper initialization
es = EarlyStopping(patience=5)

# LR scheduler initialization
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)

# Initialize datasets and dataloaders
train_dataset = TrainValidationDataset("/content/train_dataset_for_segmentation_model.csv", "", [1, 2, 3, 4, 5, 6, 7], transform=image_transformation, crop_transform=image_transform_with_bbox_crop, crop_to_bbox_detections=True)
validation_dataset = TrainValidationDataset("/content/validation_dataset_for_segmentation_model.csv", "", [1, 2, 3, 4, 5, 6, 7], transform=image_transformation, crop_transform=image_transform_with_bbox_crop, crop_to_bbox_detections=True)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=2, shuffle=True)

# Gather data for graphing
train_loss_arr = []
test_loss_arr = []

# Train the model
n_total_steps_train = len(train_dataloader)
n_total_steps_validation = len(validation_dataloader)
# Data structure to hold the best values
best_model_on_loss_value = {'train_loss':1000000, 'test_loss':1000000, 'epoch':0}
best_model_on_accuracy_value = {'train_loss':1000000, 'test_loss':1000000, 'epoch':0}
for epoch in range(num_epochs):
    avg_train_loss = 0
    for i, (imgs1, imgs2, labels) in enumerate(train_dataloader):
        imgs1, imgs2, labels = imgs1.to('cuda'), imgs2.to('cuda'), labels.to('cuda')

        # Forward pass
        embs1, embs2 = model(imgs1, imgs2)
        loss = criterion(embs1, embs2, labels)

        print(f'Epoch {epoch+1} Batch {i + 1}, Training Loss: {loss:.6f}')

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_train_loss += loss.item()
    print('\n============================================================================================')
    print(f'Epoch {epoch+1}, Training Loss: {(avg_train_loss / n_total_steps_train):.6f}')
    print('============================================================================================\n')

    # Validate the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        avg_val_loss = 0
        n_correct = 0
        n_samples = 0
        for imgs1, imgs2, labels in validation_dataloader:
            imgs1, imgs2, labels = imgs1.to('cuda'), imgs2.to('cuda'), labels.to('cuda')

            # Forward pass
            embs1, embs2 = model(imgs1, imgs2)
            loss = criterion(embs1, embs2, labels)

            avg_val_loss += loss

        scheduler.step(avg_val_loss / n_total_steps_validation)

        print('\n============================================================================================')
        print(f'Epoch {epoch+1}, Validation Loss: {(avg_val_loss / n_total_steps_validation):.6f}')
        print('============================================================================================\n')

        train_loss_arr.append(avg_train_loss / n_total_steps_train)
        test_loss_arr.append(avg_val_loss / n_total_steps_validation)

        if es(model, (avg_val_loss / n_total_steps_validation)):
            print('Early stopping criteria reached. Training stopped.')
            break

Epoch 1 Batch 1, Training Loss: 171.729279


OutOfMemoryError: ignored

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi